# Multi-label Emotion Detection

SemEval2025 Task 11 Track A + C

Finetune model to generate code
use our data to fine-tune and evaluate
Evaluate fine-tuned vs zero-shot vs baseline (generate n times and record error rate)
Use a dataset for training

## 1A. Environment Set-up 

In [2]:
# Environment check 
import os
assert os.environ['CONDA_DEFAULT_ENV'] == "cs375"

import sys
assert sys.version_info.major == 3 and sys.version_info.minor == 11

#### Installing other packages

In [3]:
import re
import typing
from typing import List
import numpy as np
import torch
import torch.nn.functional as F
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer, DataCollatorWithPadding)
from datasets import Dataset, load_dataset
from sklearn.metrics import f1_score
import pandas as pd

## 1B. Data exploration

In [5]:
# Load the SemEval dataset
file_path = "public_data/train/track_a/eng.csv"
data = pd.read_csv(file_path)
emotion_columns = ["Anger", "Fear", "Joy", "Sadness", "Surprise"]
print(data.head())

                        id                                               text  \
0  eng_train_track_a_00001                                But not very happy.   
1  eng_train_track_a_00002  Well she's not gon na last the whole song like...   
2  eng_train_track_a_00003  She sat at her Papa's recliner sofa only to mo...   
3  eng_train_track_a_00004                    Yes, the Oklahoma city bombing.   
4  eng_train_track_a_00005                       They were dancing to Bolero.   

   Anger  Fear  Joy  Sadness  Surprise  
0      0     0    1        1         0  
1      0     0    1        0         0  
2      0     0    0        0         0  
3      1     1    0        1         1  
4      0     0    1        0         0  
